# Price-taker with uncertainty analysis notebook

In [1]:
import numpy as np
import pandas as pd
from numbers import Real
from pathlib import Path
from functools import partial
# from Backcaster_of_price_taker import PricetakerBackcaster
from Backcaster_of_price_taker import PriceBackcaster
from dispatches.case_studies.renewables_case.load_parameters import *

In [2]:
lmps_df = pd.read_parquet(Path("data", "303_LMPs_15_reserve_500_shortfall.parquet"))
lmps = lmps_df['LMP'].values
lmps[lmps>500] = 500

In [3]:
def check_two_signals(signal_1, signal_2):
    return np.allclose(signal_1, signal_2)

In [11]:
# define the initial values
signal = lmps
horizon = 24
future_horizon = 48
scenario = 3

pb = PriceBackcaster(signal, scenario=scenario, pointer=0, horizon=horizon, future_horizon=future_horizon)

real_signal = lmps[0:24]
fetched_signal = pb.generate_stage_1_price_signals()
check_two_signals(real_signal, fetched_signal)

True

In [16]:
real_signal_1 = lmps[-48:]
fetched_signal_1 = pb.generate_stage_2_price_signals()[2]
check_two_signals(real_signal_1, fetched_signal_1)

True

In [18]:
real_signal_2 = lmps[-72:-24]
fetched_signal_2 = pb.generate_stage_2_price_signals()[1]
check_two_signals(real_signal_2, fetched_signal_2)

True

In [20]:
real_signal_3 = lmps[-96:-48]
fetched_signal_3 = pb.generate_stage_2_price_signals()[0]
check_two_signals(real_signal_3, fetched_signal_3)

True

In [22]:
pb.pointer = 10

real_signal = lmps[10*24: 10*24+24]
fetched_signal = pb.generate_stage_1_price_signals()
check_two_signals(real_signal, fetched_signal)

True

In [24]:
real_signal_1 = lmps[(pb.pointer-2)*24:pb.pointer*24]
fetched_signal_1 = pb.generate_stage_2_price_signals()[2]
check_two_signals(real_signal_1, fetched_signal_1)

True

In [25]:
real_signal_2 = lmps[(pb.pointer-3)*24:(pb.pointer-1)*24]
fetched_signal_2 = pb.generate_stage_2_price_signals()[1]
check_two_signals(real_signal_2, fetched_signal_2)

True

In [26]:
real_signal_3 = lmps[(pb.pointer-4)*24:(pb.pointer-2)*24]
fetched_signal_3 = pb.generate_stage_2_price_signals()[0]
check_two_signals(real_signal_3, fetched_signal_3)

True

## Inspect the test results

In [26]:
import json
path = "test_wb_new_uncertainty_scenario3_24_48.json"
with open(path, 'rb') as f:
    res_dict = json.load(f)

res_dict['0']["wind_gen"]

[826.2000081924941,
 819.2000079614942,
 775.4000078954942,
 749.5000071254942,
 802.1000077964942,
 819.5000085884942,
 794.8000077854941,
 761.9000076314942,
 743.5000072904942,
 488.3000049914941,
 247.90000228549414,
 140.0000012294941,
 131.90000124287386,
 65.70000094587493,
 45.800000857860155,
 122.80000085549402,
 115.20000106687735,
 205.80000238690886,
 360.10000386089587,
 308.5000027388944,
 309.9000031238885,
 287.9000028818829,
 170.40000208989332,
 99.0000010888885]

In [27]:
for key in res_dict.keys():
    print(res_dict[key]['solver_stat'])

optimal
optimal
optimal


In [28]:
rev=0
for key in res_dict.keys():
    rev += res_dict[key]["revenue"][0]
#     print(res_dict[key]['stage_1_start_soc'], res_dict[key]['stage_1_end_soc'])
#     print(res_dict[key]['stage_2_start_soc'], res_dict[key]['stage_2_end_soc'])
#     print("--------------------------------------------------------------------")
print(rev)

2063431.3728448213


In [29]:
for i in range(10):
    print(f"day_{i}  ", lmps[i*24:(i+1)*24])
    print("----------------------------------")

day_0   [ 0.        0.        0.        0.        0.        0.        0.
 -0.        0.        0.        0.        0.       18.861019 19.034775
  0.        0.       19.430296 26.486943 23.069973 22.732362 21.474047
 20.419098 22.492854 21.474047]
----------------------------------
day_1   [21.287879 72.185841 72.492854 72.96816  71.843654 20.419098 21.287879
 73.069973  0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.       -0.
 -0.        0.       -0.      ]
----------------------------------
day_2   [ 0.        0.        0.        0.       19.68943  -0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.       18.861019 19.68943  22.576747
 23.069973 23.128644 22.576747]
----------------------------------
day_3   [23.128644 23.128644 23.069973 23.069973 22.96816  23.128644 24.617414
 76.324557 22.492854 19.034775  0.        0.        0.        0.
  0.        0.        

In [30]:
from load_parameters import default_input_params
default_input_params.keys()
default_input_params['wind_resource']
s = 0
for i in default_input_params['wind_resource']:
    s += default_input_params['wind_resource'][i]['wind_resource_config']['capacity_factor'][0]*847
s

2069625.1333333408